In [ ]:
from quickdb.datarake import master
from tqdm import tqdm

### helper functions

In [ ]:
def getbody(func):
    import inspect
    return '\n'.join(inspect.getsource(func).split('\n')[1:])

In [ ]:
import contextlib

@contextlib.contextmanager
def progress_bar():
    with tqdm(total=1, ncols=80) as pbar:
        def progress(p):
            pbar.total = p.total
            pbar.n = p.done
            pbar.refresh()
        yield progress

### 1D histogram

In [ ]:
def make_env():
    import numpy

    rerun = 'pdr2_wide'

    def mapper(patch):
        mag = patch('forced.i.psfflux_mag')
        return numpy.histogram(mag, bins=50, range=(0, 30))

    def reducer(a, b):
        return a[0] + b[0], a[1]

with progress_bar() as progress:
    %time result = master.run_make_env(getbody(make_env), {}, progress)

%matplotlib inline
from matplotlib import pyplot

hist, bins = result
pyplot.plot(bins[1:], hist)
pyplot.grid()

### 2D histogram

In [ ]:
def make_env():
    import numpy
    import numexpr

    rerun = 'pdr2_wide'

    def flux2mag(flux):
         return -2.5 * numpy.log10(flux * (10**-9) / 3631.)
         # return numexpr('-2.5 * log10(flux * (10**-9) / 3631.)')

    def mapper(patch):
        patch = patch[patch('forced.isprimary') & (patch('forced.i.extendedness_value') < 0.5)]
        g = patch('forced.g.psfflux_mag')
        r = patch('forced.r.psfflux_mag')
        i = patch('forced.i.psfflux_mag')
        return numpy.histogram2d(g - r, r - i, bins=(400, 800), range=((-0.5, 2), (-0.5, 2.5)))

    def reducer(a, b):
        return a[0] + b[0], a[1], a[2]

with progress_bar() as progress:
    %time result = master.run_make_env(getbody(make_env), {}, progress)

import numpy

hist, xedges, yedges = result
pyplot.imshow(numpy.log(1 + hist).T, origin='low', extent=(xedges[0], xedges[-1], yedges[0], yedges[-1]))

### SQL interface

In [ ]:
from quickdb.sql2mapreduce import run_sql

In [ ]:
sql = '''
    SELECT
        COUNT(*)
    FROM
        pdr2_wide
    GROUP BY
        forced.isprimary
'''

with progress_bar() as progress:
    %time result = run_sql(sql, master.run_make_env, progress=progress)


from collections import OrderedDict
import pandas

pandas.DataFrame.from_dict(OrderedDict(zip(result.target_names, result.target_list)))

In [ ]:
sql = '''
    SELECT
        histogram2d(
            forced.g.psfflux_mag - forced.r.psfflux_mag,
            forced.r.psfflux_mag - forced.i.psfflux_mag,
            bins => (400, 800),
            range => ((-0.5, 2.), (-0.5, 2.5))
        )
    FROM
        pdr2_wide
    WHERE
        forced.i.extendedness_value < 0.5
'''

with progress_bar() as progress:
    %time result = run_sql(sql, master.run_make_env, progress=progress)

hist, xedges, yedges = result.target_list[1][0]
pyplot.imshow(numpy.log(1 + hist).T, origin='low', extent=(xedges[0], xedges[-1], yedges[0], yedges[-1]))

### Cross match

In [ ]:
import hscmap

In [ ]:
import numpy

def gen_coord(n):
    a0 = 330
    a1 = 370
    d0 = -1
    d1 = +2.5
    r = numpy.random.uniform(a0, a1, n)
    d = numpy.random.uniform(d0, d1, n)
    return numpy.array([r, d])

my_cat = gen_coord(10000)

In [ ]:
w = hscmap.Window()

In [ ]:
w.catalogs.new(*my_cat, color=[0, 1, 0, 0.25])

In [ ]:
sql = f'''
SELECT
    crossmatch(
        forced.coord,             -- reference catalog
        shared.my_cat / degree,   -- user's catalog [radian]
        5 / arcsec,               -- match radius   [radian]
        object_id,
        forced.coord_ra * degree,
        forced.coord_dec * degree
    )
FROM
    pdr2_wide
WHERE
    forced.isprimary
'''

with progress_bar() as progress:
    %time result = run_sql(sql, master.run_make_env, shared={'my_cat': my_cat}, progress=progress)

crossmatch = result.target_list[1][0]

object_id, ra, dec = crossmatch.fields

c2 = w.catalogs.new(ra, dec , color=[1,0,0,0.25])

In [ ]:
c2.remove()